In [1]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd
import re

In [2]:
def get_soup(url):
    t = 1
    #If the url is either a product page or a page within category (i.e. a slow-loading page), wait longer
    if url.startswith("https://www.amazon.com/dp/") or url.startswith("https://www.amazon.com/s?i"):
        time_wait = 20
    #else it's a review page and those load faster
    else:
        time_wait = 3
        #via docker
        r = requests.get("http://localhost:8050/render.html", params={"url": url,'wait': time_wait, 'images': 0})
        soup = BeautifulSoup(r.text, "html.parser")
        #error 504 is a globaltimeout error, meaning splash couldn't keep up
        if soup.text.startswith('{"error": 504'):
            raise Exception(f"splash couldn't keep up at {url}")
        elif soup.title.text == 'Amazon.com':
            raise Exception(f"blocked at url: {url}")
        else:
            print(f"page successfully loaded in time {time_wait}: {url}")       
    return soup

In [9]:
r = requests.get("http://localhost:8050/render.html", params={"url": "https://www.amazon.com/dp/B07YMNVXVV",'wait': 4, 'viewport': '320x480', 'images': 0})
soup = BeautifulSoup(r.text, "html.parser")
soup.title.text

'Amazon.com'

In [3]:
product_infos = pd.read_excel("../../data/links_over_500_reviews.xlsx")
product_infos

,link,brand,model,title
0,https://www.amazon.com/Apple-iPhone-XR-Fully-U...,apple,Iphone XR,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)"
1,https://www.amazon.com/Apple-iPhone-XR-Fully-U...,apple,Iphone XR,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)"
2,https://www.amazon.com/Apple-iPhone-XR-Fully-U...,apple,Iphone XR,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)"
3,https://www.amazon.com/Apple-iPhone-XR-Fully-U...,apple,Iphone XR,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)"
4,https://www.amazon.com/Apple-iPhone-XR-Fully-U...,apple,Iphone XR,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)"
...,...,...,...,...
201,https://www.amazon.com/Samsung-Galaxy-SM-G9650...,samsung,Galaxy S9 Plus,"Samsung Galaxy S9+, 64GB, Midnight Black - Ful..."
202,https://www.amazon.com/Samsung-Factory-Unlocke...,samsung,Galaxy S10,"Samsung Galaxy S10, 128GB, Prism Black - Unloc..."
203,https://www.amazon.com/Xiaomi-Note-11-Unlocked...,xiaomi,NOTE 11,Xiaomi Redmi Note 11 4G Volte 128GB + 4GB Fact...
204,https://www.amazon.com/Samsung-Galaxy-Unlocked...,samsung,Galaxy Note 10+,"Samsung Galaxy Note 10+, 256GB, Aura Glow - Fo..."


In [4]:
data_so_far = pd.read_csv("../../data/amazon_500+.csv", sep = ";", header = None)
data_so_far.columns = ["asin_url", "brand", "model", "link", "product_title",
             "name", "rating", "review_title", "country", "date",
             "specs", "first_spec", "review", "helpful", "picture",
            "verified", "profile", "video"]
data_so_far
data_so_far.asin_url.unique()

array(['https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07T3TJ8F6/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=1',
       'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07P978C2R/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=1',
       'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07P978C2R/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=2',
       ...,
       'https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=302',
       'https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=303',
       'https://www.amazon.c

In [5]:
'='.join(data_so_far.asin_url.unique()[0].split("=")[:-1])+"="
unique_links = []
for link in data_so_far.asin_url.unique():
    unique_link = '='.join(link.split("=")[:-1])+"="
    unique_links.append(unique_link)
unique_links

['https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07T3TJ8F6/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=',
 'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07P978C2R/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=',
 'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B07P978C2R/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=',
 'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B08BGD4G36/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=',
 'https://www.amazon.com/Apple-iPhone-XR-Fully-Unlocked/product-reviews/B08BGD4G36/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&formatType=current_format&pageNumber=',
 'https://www.amazon.com/Apple-iPhone-XR-Full

In [8]:
all_reviews = []
missed_asins = []
num_row = 1
for index, row in product_infos[200:202].iterrows():
    print(f"{num_row}/{len(product_infos)}")
    num_row += 1
    #if row['link'] not in unique_links:
    if row['link'] == "https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=":
        product_info = row
        get_reviews_from_page(product_info)

1/206
2/206
succesfully loaded Samsung Galaxy S9+, 64GB, Midnight Black - Fully Unlocked (Renewed)
page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_getr_d_paging_btm_next_305?pageNumber=305
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
10
page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_getr_d_paging_btm_next_306?pageNumber=306
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: 

page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_getr_d_paging_btm_next_322?pageNumber=322
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
180
page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_getr_d_paging_btm_next_323?pageNumber=323
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
190
page successfully loaded in time 3: 

page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_getr_d_paging_btm_next_338?pageNumber=338
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
340
page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_getr_d_paging_btm_next_339?pageNumber=339
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['help

page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_getr_d_paging_btm_next_352?pageNumber=352
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
480
page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_getr_d_paging_btm_next_353?pageNumber=353
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['help

page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_getr_d_paging_btm_next_368?pageNumber=368
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
640
page successfully loaded in time 3: https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_getr_d_paging_btm_next_369?pageNumber=369
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['helpful']
the following items were missing: ['help

In [6]:
#This function retrieves all the review info from a page
def get_reviews_from_page(product_info):
    print(f"succesfully loaded {product_info['title']}")
    with open(f"../../data/amazon_500+.csv", "a", encoding = "UTF-8", newline = '') as csv_file:
        writer = csv.writer(csv_file, delimiter = ";")
        for i in range(305,501):
            soup_reviews = get_soup((product_info['link']+str(i)).replace("ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=", f"ref=cm_cr_getr_d_paging_btm_next_{i}?pageNumber="))
            reviews = soup_reviews.find_all('div', {"data-hook":"review"})
            if len(reviews) == 0:
                print(f'Could not find any reviews at page {i}')
                break
            else:
                pass
            for review in reviews:
                review_info = {}
                review_info = {
                'product_url' : product_info['link']+str(i),
                'brand' : product_info['brand'],
                'model' : product_info['model'],
                'link' : product_info['link']+str(i),
                'product_title' : product_info['title']
                }
                try:
                    review_info['name'] = review.find('span', class_="a-profile-name").text
                except:
                    review_info['name'] = None
                try:
                    review_info['star_rating'] = float(review.find('i', {"data-hook":"review-star-rating"}).text.replace(' out of 5 stars',''))
                except:
                    #foreign countries
                    try:
                        review_info['star_rating'] = float(review.find('i', {"data-hook":"cmps-review-star-rating"}).text.replace(' out of 5 stars',''))
                    except:
                        review_info['star_rating'] = None
                try:
                    review_info['title'] = review.find('a', {"data-hook":"review-title"}).text.strip()
                except:
                    try:
                        #foreign countries
                        review_info['title'] = review.find('span',{"data-hook":"review-title"}).text.strip()
                    except:
                        review_info['title'] = None
                try:
                    review_info['country'] = review.find('span', {"data-hook":"review-date"}).text.replace('Reviewed in ','').rpartition(' on')[0]
                except:
                    review_info['country'] = None
                try:
                    review_info['date'] = review.find('span', {"data-hook":"review-date"}).text.replace('Reviewed in ','').rpartition(' on')[2].strip()
                except:
                    review_info['date'] = None
                attr = []
                try:
                    attributes = str(reviews[0].find('a', {"data-hook":"format-strip"}))
                    for item in re.findall(r'[>](.+?)[<]', attributes):
                        attr.append(item.replace('</i>',''))
                    review_info['info'] = attr
                except:
                    review_info['info'] = None
                try:
                    for at in attr:
                        if at.find("Color:") != -1:
                            color = at.replace("Color: ", "")
                    review_info['color'] = color
                except:
                    review_info['color'] = None
                try:
                    review_info['review'] = review.find('span', {"data-hook":"review-body"}).text.strip()
                except:
                    review_info['review'] = None
                try:
                    helpful = review.find('span', {"data-hook" : "helpful-vote-statement"}).text.strip()
                    if helpful == "One person found this helpful":
                        review_info['helpful'] = 1
                    elif "found this helpful" in helpful:
                        review_info['helpful'] = helpful.split()[0]
                    else:
                        review_info['helpful'] = helpful
                except:
                    review_info['helpful'] = None
                try:
                    if review.find('div', class_="review-image-tile-section") is None:
                        review_info['picture'] = 'no'
                    else:
                        review_info['picture'] = len(review.find('div', class_="review-image-tile-section").find_all('span'))
                except:
                    review_info['picture'] = None

                try:
                    if review.find('span', {"data-hook":"avp-badge"}) is None:
                        review_info['verified'] = 'no'
                    else:
                        review_info['verified'] = 'yes'
                except:
                    review_info['verified'] = None
                    
                try:
                    picture = review.find('div', class_="a-profile-avatar").find('noscript').text
                    if picture.find('default') != -1:
                        review_info['profile_picture'] = 'no'
                    else:
                        review_info['profile_picture'] = 'yes'
                except:
                    review_info['profile_picture'] = None
                    
                try:
                    video = review.find('div', class_="vse-airy-container vse-player-container")
                    if video == None:
                        review_info['video'] = 'no'
                    else:
                        review_info['video'] = 'yes'
                except:
                    review_info['video'] = None

                all_reviews.append(review_info)
                missing_values = [key for key,value in review_info.items() if value == None]
                if len(missing_values) > 0:
                    print(f"the following items were missing: {missing_values}")
                writer.writerow(list(review_info.values()))
            print(len(all_reviews))
            if not soup_reviews.find('li', class_ = "a-disabled a-last"):
                pass
            else:
                print(f"page {i} was the last page")
                break